# Обработка текстовых данных

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
folder_path = '/content/drive/MyDrive/СРОД/data/text/'

In [8]:
data = pd.read_csv(folder_path+'text.csv')
data

,text,emo
0,о как,удивление
1,"ну да, ну да, пошел я нахер",сарказм
2,"моя главная профессия повар, хааа. а твоя? ха-...",радость
3,"я не уступаю место бабкам в метро, потому что ...",мотивация
4,"сука, блять, я тебя захуярю, понял? козлина еб...",гнев
5,"Я тоже может хочу шоколад блин, я тоже хочу шо...",грусть


In [4]:
import sys
import requests
from pymystem3 import Mystem
m = Mystem()

def tag_mystem(
    text="Текст нужно передать функции в виде строки!", mapping=None, postags=True
):
    # если частеречные тэги не нужны (например, их нет в модели), выставьте postags=False
    # в этом случае на выход будут поданы только леммы

    processed = m.analyze(text)
    tagged = []
    for w in processed:
      if "analysis" in w and w["analysis"]:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(",")[0]
            pos = pos.split("=")[0].strip()
            if mapping:
                if pos in mapping:
                    pos = mapping[pos]  # здесь мы конвертируем тэги
                else:
                    pos = "X"  # на случай, если попадется тэг, которого нет в маппинге
            tagged.append(lemma.lower() + "_" + pos)
        except KeyError:
            continue  # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    if not postags:
        tagged = [t.split("_")[0] for t in tagged]
    return tagged

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [5]:
# Таблица преобразования частеречных тэгов Mystem в тэги UPoS:
mapping_url = "https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map"

mystem2upos = {}
r = requests.get(mapping_url, stream=True)
for pair in r.text.split("\n"):
    pair = pair.split()
    if len(pair) > 1:
        mystem2upos[pair[0]] = pair[1]


In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def text_prepare(text):
  filtered_word_list = []
  res = str(text).strip()
  list_of_normal_words = tag_mystem(text=res, mapping=mystem2upos)
  for word in list_of_normal_words:
    if word not in stop_words:
        filtered_word_list.append(word)
  return list_of_normal_words

In [9]:
data["word_in_normal_view_with_tags"] = data["text"].progress_apply(text_prepare)

100%|██████████| 6/6 [00:00<00:00,  7.83it/s]


In [10]:
def text_prepare_without_tags(text):
  filtered_word_list = []
  res = str(text).strip()
  list_of_normal_words = tag_mystem(text=res, mapping=mystem2upos, postags=False)
  for word in list_of_normal_words:
    if word not in stop_words:
        filtered_word_list.append(word)
  return filtered_word_list

In [11]:
data["word_in_normal_view_without_tags_and_stopw"] = data["text"].progress_apply(text_prepare_without_tags)

100%|██████████| 6/6 [00:00<00:00, 1092.27it/s]


In [12]:
data

,text,emo,word_in_normal_view_with_tags,word_in_normal_view_without_tags_and_stopw
0,о как,удивление,"[о_INTJ, как_ADV]",[]
1,"ну да, ну да, пошел я нахер",сарказм,"[ну_PART, да_PART, ну_PART, да_SCONJ, пойти_VE...","[пойти, нахер]"
2,"моя главная профессия повар, хааа. а твоя? ха-...",радость,"[мой_DET, главный_ADJ, профессия_NOUN, повар_N...","[главный, профессия, повар, хааа, твой, ха-ха,..."
3,"я не уступаю место бабкам в метро, потому что ...",мотивация,"[я_PRON, не_PART, уступать_VERB, место_NOUN, б...","[уступать, место, бабки, метро, бабка, жизнь, ..."
4,"сука, блять, я тебя захуярю, понял? козлина еб...",гнев,"[сука_NOUN, блять_VERB, я_PRON, ты_PRON, захуя...","[сука, блять, захуяривать, понимать, козлиный,..."
5,"Я тоже может хочу шоколад блин, я тоже хочу шо...",грусть,"[я_PRON, тоже_PART, мочь_VERB, хотеть_VERB, шо...","[мочь, хотеть, шоколад, блин, хотеть, шоколадка]"


In [13]:
data.to_csv(folder_path+"text_final.csv", encoding='utf-8')